In [1]:
from gee_scripts.get_sources import get_s1_image, get_gldas, get_gpm, get_hansen
from gee_scripts.get_sources import get_srtm, get_globcover, get_gedi, get_gldas_stats


In [ ]:
import ee
ee.Initialize()

In [2]:
date = ee.Date("2023-01-01")
aoi = ee.Geometry.Polygon(

In [ ]:
def get_sources(date):
    
    date_range = ee.Date(date_feature.get("date")).getRange('day')

    s1_composite = get_s1_image(date_range, ee_region)

    return s1_composite.set({
        "numberOfBands" : s1_composite.bandNames().size(),
        "date" : ee.Date(date_feature.get("date"))
        })
    
def reduce_composite(composite):
    
    # Filter the extra data with the matching date
    date = composite.get("date")
    date_range = ee.Date(date).getRange('day')

    gldas_composite = get_gldas(date_range, ee_region)
    gpm_composite = get_gpm(date_range, ee_region)

    composite = (ee.Image(composite)
        .addBands(gldas_composite)
        .addBands(gpm_composite)
    )
    
    return composite.reduceRegions(**{
        "collection" : ee_points,
        "reducer" : ee.Reducer.first(),
        "scale" : 10,
        "tileScale" : 16
    }).filter(ee.Filter.notNull(['VH'])).map(lambda feature: feature.set({
        "date" : date
    }))


In [ ]:
task = (ee_dates
        .map(get_sources)
        .filter(ee.Filter.gt('numberOfBands', 0))
        .map(reduce_composite).flatten()
)